`Note`
- case_id 가 중복되는 경우 첫 번째 row 만 값이 발견됨. 
- train_person_1 (internal_data) 의 'birthdate_87D' 는 null 값이 너무 많아 의미가 없어보임. 반면 'birth_259D' 는 tarin_base 와 동일한 rows 의 개수와 case_id 를 갖고 있다.

In [1]:
import os, glob
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

PATH_DATASET = Path("../data")
PATH_PARQUETS = PATH_DATASET / "parquet_files"
PARQUETS_TRAIN = PATH_PARQUETS / "train"
PARQUETS_TEST = PATH_PARQUETS / "test"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [73]:
df_base = pl.read_parquet(PARQUETS_TRAIN / 'train_base.parquet')
df_base

case_id,date_decision,MONTH,WEEK_NUM,target
i64,str,i64,i64,i64
0,"""2019-01-03""",201901,0,0
1,"""2019-01-03""",201901,0,0
2,"""2019-01-04""",201901,0,0
3,"""2019-01-03""",201901,0,0
4,"""2019-01-04""",201901,0,1
5,"""2019-01-02""",201901,0,0
6,"""2019-01-03""",201901,0,0
7,"""2019-01-03""",201901,0,0
8,"""2019-01-03""",201901,0,0


In [2]:
def check_parquets():
    for p in glob.glob(str(PARQUETS_TRAIN / f"*.parquet")):
        df_ = pl.read_parquet(p)
        cols_with_birth = [col for col in df_.columns if 'birth' in col]
        if cols_with_birth:
            print((p.split('\\')[-1], cols_with_birth))

In [3]:
check_parquets()

('../data/parquet_files/train/train_person_1.parquet', ['birth_259D', 'birthdate_87D'])
('../data/parquet_files/train/train_static_cb_0.parquet', ['birthdate_574D', 'dateofbirth_337D', 'dateofbirth_342D'])


In [66]:
df_internal = pl.read_parquet(PARQUETS_TRAIN / 'train_person_1.parquet')
df_external = pl.read_parquet(PARQUETS_TRAIN / 'train_static_cb_0.parquet')

In [67]:
df_internal_cols = ['case_id', 'birth_259D', 'birthdate_87D']
df_external_cols = ['case_id', 'birthdate_574D', 'dateofbirth_337D', 'dateofbirth_342D']

In [68]:
df_internal = df_internal.select(pl.col(df_internal_cols))
df_external = df_external.select(pl.col(df_external_cols))

In [69]:
display(df_internal.null_count())

# birthdate_87D 은 거의다 Null 값이므로 제외하는게 좋을 듯함
df_internal = df_internal.select(pl.col('case_id', 'birth_259D'))

case_id,birth_259D,birthdate_87D
u32,u32,u32
0,1447332,2949075


In [65]:
df_internal

# null 값을 처리하고 싶은데, 특이한 특징을 보임.
# 중복되는 case_id 의 첫 번째 데이터만 null 값이 아님
# 그게 맞는지 모든 rows 에 대해서 확인할 필요가 있음.

case_id,birth_259D
i64,str
0,"""1986-07-01"""
0,null
0,null
0,null
1,"""1957-08-01"""
1,null
1,null
1,null
1,null


In [71]:
df_internal.group_by(pl.col('case_id')).first().null_count()

case_id,birth_259D
u32,u32
0,0


In [77]:
df_internal = df_internal.group_by(pl.col('case_id')).first().sort(by='case_id')
df_internal
## tarin base 와 동일한 rows 개수임

case_id,birth_259D
i64,str
0,"""1986-07-01"""
1,"""1957-08-01"""
2,"""1974-12-01"""
3,"""1993-08-01"""
4,"""1994-01-01"""
5,"""1979-10-01"""
6,"""1991-01-01"""
7,"""1993-09-01"""
8,"""1982-11-01"""


In [79]:
# df_internal와 df_train에서 'case_id' 열을 추출합니다.
case_ids_internal = df_internal['case_id']
case_ids_base = df_base['case_id']

# 각 'case_id' 열의 고유한 값들을 확인합니다.
unique_case_ids_internal = set(case_ids_internal.to_list())
unique_case_ids_train = set(case_ids_base.to_list())

# 두 데이터프레임의 'case_id' 열의 고유한 값이 서로 일치하는지 확인합니다.
if unique_case_ids_internal == unique_case_ids_train:
    print(True)
else:
    print(False)

True


`external`

In [80]:
df_external
## train_base 보다 부족한 Row 개수를 갖는다.

case_id,birthdate_574D,dateofbirth_337D,dateofbirth_342D
i64,str,str,str
357,"""1988-04-01""",null,null
381,"""1973-11-01""",null,null
388,"""1989-04-01""","""1989-04-01""",null
405,"""1974-03-01""","""1974-03-01""",null
409,"""1993-06-01""","""1993-06-01""",null
410,"""1978-06-01""","""1978-06-01""",null
411,"""1959-07-01""","""1959-07-01""",null
412,"""1969-01-01""",null,null
413,"""1955-12-01""",null,null


In [83]:
display(df_external.null_count() / len(df_external))
df_external = df_external.select(pl.exclude('dateofbirth_342D'))
# dateofbirth_342D 는 null data 가 너무 많아 제외.

case_id,birthdate_574D,dateofbirth_337D,dateofbirth_342D
f64,f64,f64,f64
0.0,0.594881,0.076499,0.975674


In [84]:
df_external

case_id,birthdate_574D,dateofbirth_337D
i64,str,str
357,"""1988-04-01""",null
381,"""1973-11-01""",null
388,"""1989-04-01""","""1989-04-01"""
405,"""1974-03-01""","""1974-03-01"""
409,"""1993-06-01""","""1993-06-01"""
410,"""1978-06-01""","""1978-06-01"""
411,"""1959-07-01""","""1959-07-01"""
412,"""1969-01-01""",null
413,"""1955-12-01""",null


In [90]:
case_ids_internal = df_internal['case_id'].to_list()
case_ids_external = df_external['case_id'].to_list()

unique_case_ids_internal = set(case_ids_internal)
unique_case_ids_external = set(case_ids_external)

if unique_case_ids_external.issubset(unique_case_ids_internal):
    print(True)
else:
    print(False)

True


In [91]:
df = df_internal.join(df_external, on='case_id', how='left')
df

case_id,birth_259D,birthdate_574D,dateofbirth_337D
i64,str,str,str
0,"""1986-07-01""",null,null
1,"""1957-08-01""",null,null
2,"""1974-12-01""",null,null
3,"""1993-08-01""",null,null
4,"""1994-01-01""",null,null
5,"""1979-10-01""",null,null
6,"""1991-01-01""",null,null
7,"""1993-09-01""",null,null
8,"""1982-11-01""",null,null


In [111]:
df.with_columns(
    [
        (pl.col('birth_259D').str.to_datetime().dt.date()),
        (pl.col('birthdate_574D').str.to_datetime().dt.date()),
        (pl.col('dateofbirth_337D').str.to_datetime().dt.date()),
    ]
)        

case_id,birth_259D,birthdate_574D,dateofbirth_337D
i64,date,date,date
0,1986-07-01,null,null
1,1957-08-01,null,null
2,1974-12-01,null,null
3,1993-08-01,null,null
4,1994-01-01,null,null
5,1979-10-01,null,null
6,1991-01-01,null,null
7,1993-09-01,null,null
8,1982-11-01,null,null
